In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE140161"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE140161"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE140161.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE140161.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE140161.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Systems biology demonstrates the predominant role of circulating interferon-alpha in primary Sjögren's syndrome and a genetic association with the class II HLA DQ locus"
!Series_summary	"Primary Sjögren’s syndrome (pSS) is the second most frequent systemic autoimmune disease, affecting 0.1% of the general population. No specific immunomodulatory drug has demonstrated efficacy for this disease, and no biomarker is available to identify patients at risk of developing systemic complications. To characterize the molecular and clinical variability across pSS patients, we integrated transcriptomic, proteomic, cellular and genetic data with clinical phenotypes in a cohort of 351 pSS patients. Unbiased global transcriptomic analysis revealed an IFN gene signature as the strongest driver of transcriptomic variability. The resulting stratification was replicated in three independent cohorts. As transcriptomic analysis did not discriminate between type I and

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data from Affymetrix chip
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: All patients have "Sjögren's syndrome", which is not the trait of interest (RA)
trait_row = None  # The trait we're looking for (Rheumatoid_Arthritis) is not available

# For gender: Found in row 1
gender_row = 1

# For age: Not available in the sample characteristics
age_row = None  

# 2.2 Data Type Conversion

def convert_trait(value: str) -> Optional[int]:
    # We don't need this function as trait data is not available
    # But we'll define it for completeness
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if "Sjögren's syndrome" in value:
        return 1  # This would be 1 if we were studying Sjögren's syndrome
    return None

def convert_gender(value: str) -> Optional[int]:
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

def convert_age(value: str) -> Optional[float]:
    # We don't need this function as age data is not available
    # But we'll define it for completeness
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 3. Save Metadata
# Perform initial filtering on the usability of the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['23064070', '23064071', '23064072', '23064073', '23064074', '23064075',
       '23064076', '23064077', '23064078', '23064079', '23064080', '23064081',
       '23064083', '23064084', '23064085', '23064086', '23064087', '23064088',
       '23064089', '23064090'],
      dtype='object', name='ID')


/media/techt/DATA/GenoAgent/tools/preprocess.py:149: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=skip_rows, comment='!', delimiter='\t',


### Step 4: Gene Identifier Review

In [5]:
# Looking at the indices of the gene expression data
# These appear to be numeric identifiers (23064070, 23064071, etc.), not standard human gene symbols
# Standard human gene symbols typically look like BRCA1, TNF, IL6, etc.
# These numeric identifiers are likely probe IDs or some other platform-specific identifiers
# that need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 10

### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the gene expression data indices and gene annotation columns
# The gene expression data has indices like '23064070', '23064071', etc.
# We need to find the matching column in the gene annotation dataframe

# In the annotation dataframe, 'ID' appears to be a probe identifier format (like 'TC0100006437.hg.1')
# However, the gene expression data uses different identifiers
# We need to determine which column matches our expression data indices

# Examining the first few gene expression indices to see their format
print("First few gene expression indices:")
print(gene_data.index[:5])

# Check if there's a direct match with any column
# Now let's look at another approach - from the annotation preview, the 'SPOT_ID.1' column 
# contains gene information like RefSeq IDs, gene names, etc.
# Let's extract human gene symbols from this field

# First, let's define the mapping columns
probe_id_col = 'ID'  # This is the probe identifier column in the annotation
gene_symbol_col = 'SPOT_ID.1'  # This contains gene information including symbols

# 2. Get gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)

# Print the mapping dataframe to observe its structure
print("\nGene mapping preview:")
print(gene_mapping.head())

# Check how many probes we have in our mapping
print(f"\nNumber of probes in mapping: {len(gene_mapping)}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many relationships between probes and genes
print("\nApplying gene mapping...")
# The gene_data index currently uses numbers, but our mapping uses probe IDs
# We need to align our data

# Check if the gene expression data indices match with our mapping
# If they don't match directly, we need to investigate further
indices_in_mapping = sum(1 for idx in gene_data.index if idx in gene_mapping['ID'].values)
print(f"Number of expression data indices found in mapping: {indices_in_mapping} out of {len(gene_data.index)}")

# If the indices don't directly match, let's try a different approach
# The issue is that we need to map between the numerical indices in gene_data and the 
# probe IDs in the annotation

# Let's create a new mapping for the probe IDs that will work with our expression data
# For this dataset, we will extract gene symbols from the annotation data and 
# normalize gene symbols in the index

# Apply gene mapping to get gene-level expression data
gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)

# Display the resulting gene expression data
print("\nGene expression data preview:")
print(gene_data.shape)
print(gene_data.head())


First few gene expression indices:
Index(['23064070', '23064071', '23064072', '23064073', '23064074'], dtype='object', name='ID')



Gene mapping preview:
                  ID                                               Gene
0  TC0100006437.hg.1  NM_001005484 // RefSeq // Homo sapiens olfacto...
1  TC0100006476.hg.1  NM_152486 // RefSeq // Homo sapiens sterile al...
2  TC0100006479.hg.1  NM_198317 // RefSeq // Homo sapiens kelch-like...
3  TC0100006480.hg.1  NM_001160184 // RefSeq // Homo sapiens pleckst...
4  TC0100006483.hg.1  NM_005101 // RefSeq // Homo sapiens ISG15 ubiq...

Number of probes in mapping: 27189

Applying gene mapping...
Number of expression data indices found in mapping: 27189 out of 27189



Gene expression data preview:
(19975, 351)
         GSM4155114  GSM4155115  GSM4155116  GSM4155117  GSM4155118  \
Gene                                                                  
A1BG       0.422857    0.378571    0.370714    0.392143    0.367857   
A1CF       0.158182    0.156364    0.167273    0.148636    0.152273   
A2M        0.327647    0.332353    0.310588    0.358235    0.351176   
A2ML1      0.209444    0.217222    0.194444    0.185000    0.200000   
A3GALT2    1.032000    0.878000    0.938000    1.006000    0.916000   

         GSM4155119  GSM4155120  GSM4155121  GSM4155122  GSM4155123  ...  \
Gene                                                                 ...   
A1BG       0.392143    0.395714    0.374286    0.387857    0.380714  ...   
A1CF       0.146818    0.152727    0.158636    0.164091    0.155455  ...   
A2M        0.298235    0.339412    0.362353    0.360588    0.344706  ...   
A2ML1      0.185000    0.190556    0.203889    0.196667    0.188333  ...   
A3

### Step 7: Data Normalization and Linking

In [8]:
# 1. We cannot process clinical features since trait_row is None (no RA data in this dataset)
# This means we're skipping the clinical feature extraction step as specified in the instructions

# Save the gene data that we've already normalized
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since there's no trait data in this dataset, we cannot link clinical and genetic data
# We'll create empty placeholder data to properly finish the process
print("No trait data available for linking with gene expression data.")

# Create a small placeholder dataframe with proper structure
empty_clinical_df = pd.DataFrame(columns=[trait])
# Use first few samples from gene_data for consistent structure
sample_ids = gene_data.columns[:5] if len(gene_data.columns) >= 5 else gene_data.columns
empty_clinical_df = pd.DataFrame(index=sample_ids, columns=[trait])
empty_clinical_df[trait] = None  # All None values for trait

# Skip handling missing values since we don't have valid linked data

# 5. We'll mark this dataset as not usable due to missing trait data
is_trait_biased = True  # Not actually assessed, but marked as biased since trait data is missing
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # No trait data
    is_biased=is_trait_biased, 
    df=empty_clinical_df,
    note="This dataset contains gene expression data for Crohn's disease, not Rheumatoid Arthritis"
)

# 6. Not saving linked data as it's not usable for our purposes
print("No valid linked data to save - dataset doesn't contain Rheumatoid Arthritis data")

Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE140161.csv
No trait data available for linking with gene expression data.
Abnormality detected in the cohort: GSE140161. Preprocessing failed.
No valid linked data to save - dataset doesn't contain Rheumatoid Arthritis data
